In [8]:
import pandas as pd
import sqlalchemy
import keys
import requests
import datetime

In [3]:
host = '127.0.0.1'
schema = 'gans'
user = 'root'
password = keys.password
port = 3306
con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

In [4]:
airports_df = pd.read_sql('airports',con=con)
airports_df

,city_id,icao
0,1,LTBA
1,3,EGLC
2,6,LECU
3,6,LEGT
4,7,UKKK
5,8,LIRA
6,8,LIRU
7,9,AZ-0001
8,9,UBBB
9,10,LRBS


In [92]:
querystring = {"withLeg":"true","withCancelled":"true","withCodeshared":"true","withCargo":"true",
               "withPrivate":"true","withLocation":"false"}
headers = {
	"X-RapidAPI-Key": "your+",
	"X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
}

In [9]:
today = datetime.date.today()
tomorrow = str(today + datetime.timedelta(days=1))
    
times = [["00:00","11:59"],["12:00","23:59"]]

In [88]:
icao_list = airports_df['icao'].to_list()

In [90]:
icao_list[1:2]

['EGLC']

In [93]:
flights_dict = {'ICAO':[],'departure_airport':[],'local_time':[]}
icao_list = airports_df['icao'].to_list()

for time in times:
    
    for icao in icao_list[1:2]:
        url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{icao}/{tomorrow}T{time[0]}/{tomorrow}T{time[1]}"
        response = requests.request("GET", url, headers=headers, params=querystring)
        print(response)
        flight_json = response.json()
        
        for flight in flight_json['arrivals']:
            flights_dict['ICAO'].append(icao)
            try:
                flights_dict['departure_airport'].append(flight['departure']['airport']['icao'])
            except:
                flights_dict['departure_airport'].append('unknown')
            try:
                flights_dict['local_time'].append(flight['arrival']['scheduledTimeLocal'])
            except:
                flights_dict['local_time'].append(pd.NaT)

<Response [200]>
<Response [200]>


In [80]:
flights_df = pd.DataFrame(flights_dict)

In [85]:
flights_df['local_time'] = pd.to_datetime(flights_df['local_time'])

In [86]:
flights_df.to_sql('flight',con=con,if_exists='append',index=False)

20